In [0]:
from datetime import datetime
from pyspark.sql.types import TimestampType

In [0]:
import pyspark.sql.functions as F
sparkifyDF = spark.read.json("/FileStore/tables/medium_sparkify_event_data.json")
sparkifyDF = sparkifyDF.dropDuplicates()
sparkifyDF = sparkifyDF.filter(~ F.col("userId").like(""))
#display(sparkifyDF)

In [0]:
sparkifyDF = sparkifyDF.withColumn("day", (sparkifyDF.ts / 86400000).cast("int"))
#display(sparkifyDF)

###Numericas

-itemInSession

-length

-registration

-status

-ts

-artist

-song

###No numericas

-userId

-firstName

-lastName

-gender

-location

-userAgent

-level

-page

-auth

-method

-length

###Definicion de abandono

columna page = "Cancellation Confirmation"

In [0]:
churnedDF = sparkifyDF.select("userId").filter(sparkifyDF.page.like("Cancellation Confirmation")).dropDuplicates()
stayedDF = sparkifyDF.select("userId").join(churnedDF, "userId", "left_anti").dropDuplicates()
display(stayedDF)
display(churnedDF)

userId
124
7
15
155
100010
200002
132
11
138
154


userId
125
51
54
100014
101
29
100021
87
73
3


In [0]:
temp = sparkifyDF.withColumn("dayOfWeek", F.dayofweek(F.to_date(sparkifyDF.ts.cast(TimestampType()))))
stayedDayOfWeek = temp.select("userId", "dayOfWeek").join(stayedDF, "userId", "inner")
churnedDayOfWeek = temp.select("userId", "dayOfWeek").join(churnedDF, "userId", "inner")
display(stayedDayOfWeek)
display(churnedDayOfWeek)

In [0]:
display(stayedDayOfWeek)
display(churnedDayOfWeek)

In [0]:
temp = sparkifyDF.select("userId", "ts").groupby("userId").max()
temp = temp.select("userId", F.col("max(ts)").alias("ts"))
temp = sparkifyDF.select("userId", "ts", "level").join(temp, ["userId", "ts"], "inner").drop("ts")
churnedLevel = temp.join(churnedDF, "userId", "inner").dropDuplicates()
stayedLevel = temp.join(stayedDF, "userId", "inner").dropDuplicates()
display(stayedLevel)
display(churnedLevel)

In [0]:
display(stayedLevel)
display(churnedLevel)

In [0]:
temp = sparkifyDF.select("userId", "page")
churnedPage = temp.join(churnedDF, "userId", "inner").select("page").groupBy("page").count()
stayedPage = temp.join(stayedDF, "userId", "inner").select("page").groupBy("page").count()

In [0]:
display(stayedPage)
display(churnedPage)

In [0]:
temp = sparkifyDF.select("userId", "userAgent")
churnedAgent = temp.join(churnedDF, "userId", "inner").select("userAgent").groupBy("userAgent").count()
stayedAgent = temp.join(stayedDF, "userId", "inner").select("userAgent").groupBy("userAgent").count()

In [0]:
display(stayedAgent)
display(churnedAgent)

In [0]:
temp = sparkifyDF.select("userId", "location")
churnedLocation = temp.join(churnedDF, "userId", "inner").select("location").groupBy("location").count()
stayedLocation = temp.join(stayedDF, "userId", "inner").select("location").groupBy("location").count()

In [0]:
display(stayedLocation)
display(churnedLocation)

###Variables derivadas

-Numero de canciones escuchadas X

-Numero de "thumbs up" X

-Numero de "thumbs down" X

-Proporcion "thumbs up" a "thumbs down" X

-Antiguedad X

-Promedio canciones por sesion X

-Total de amigos X

-Numero de errores X

-Numero de "downgrades" X

-Nivel actual X

-Numero de anuncios publicitarios por sesion X

-Proporcion numero de canciones a numero de anuncios 

-Abandono(variable target) X

In [0]:
def generateDFGender(df):
  temp = df.select("userId", "gender").dropDuplicates()
  temp = temp.withColumn("temp", F.when(F.col("gender").like("M"), 0).otherwise(1))
  newDF = temp.drop("gender")
  newDF = newDF.select("userId", F.col("temp").alias("gender"))
  return newDF

In [0]:
def generateDFAddPlaylistPerDay(df):
  temp = df.select("userId", "page", "day").filter(F.col("page").like("Add to Playlist"))
  newDF = temp.groupBy(["userId", "day"]).count()
  newDF = newDF.select("userId", "count").groupBy("userId").avg().dropDuplicates()
  temp = df.select("userId").dropDuplicates()
  newDF = temp.join(newDF, "userId", "left_outer")
  newDF = newDF.fillna(0).select("userId", F.col("avg(count)").alias("addPlaylistPerDayAVG"))
  return newDF

In [0]:
def generateDFAdvertsPerDay(df):
  temp = df.select("userId", "page", "day").filter(F.col("page").like("Roll Advert"))
  newDF = temp.groupBy(["userId", "day"]).count()
  newDF = newDF.select("userId", "count").groupBy("userId").avg().dropDuplicates()
  temp = df.select("userId").dropDuplicates()
  newDF = temp.join(newDF, "userId", "left_outer")
  newDF = newDF.fillna(0).select("userId", F.col("avg(count)").alias("adsPerDayAVG"))
  return newDF

In [0]:
def generateDFAge(df):
  temp = sparkifyDF.select("userId", "ts").groupby("userId").max() #Se guarda en temp el df con el usuario y su ultima timestamp
  temp = temp.select("userId", F.col("max(ts)").alias("ts")) #Se le cambia el nombre a la columna max(ts) devuelta a ts para hacer facil el join
  newDF = df.select("userId", "ts", "registration").join(temp, ["userId", "ts"], "inner").dropDuplicates() #Se hace join entre el df que llega y temp
  newDF = newDF.withColumn("age", (newDF.ts - newDF.registration) / 86400000) #A newDF se le agrega la nueva columna "age" que es la antiguedad del usuario en dias
  newDF = newDF.select("userId", "age") #Se seleccionan solo las columnas importantes
  return newDF

In [0]:
def generateDFLevel(df):
  temp = sparkifyDF.select("userId", "ts").groupby("userId").max()
  temp = temp.select("userId", F.col("max(ts)").alias("ts"))
  newDF = df.select("userId", "ts", "level").join(temp, ["userId", "ts"], "inner").dropDuplicates()
  newDF = newDF.select("userId", "level")
  temp = newDF.select("userId").filter(F.col("level").like("paid"))
  temp = temp.withColumn("level", F.lit(1))
  newDF = newDF.drop("level");
  newDF = newDF.join(temp, "userId", "left_outer")
  newDF = newDF.fillna(0)
  return newDF

In [0]:
def generateDFErrorsPerDay(df):
  temp = df.select("userId", "page", "day").filter(F.col("page").like("Error"))
  newDF = temp.groupBy(["userId", "day"]).count()
  newDF = newDF.select("userId", "count").groupBy("userId").avg().dropDuplicates()
  temp = df.select("userId").dropDuplicates()
  newDF = temp.join(newDF, "userId", "left_outer")
  newDF = newDF.fillna(0).select("userId", F.col("avg(count)").alias("errorsPerDayAVG"))
  return newDF

In [0]:
def generateDFDowngradesPerDay(df):
  temp = df.select("userId", "page", "day").filter(F.col("page").like("Downgrade"))
  newDF = temp.groupBy(["userId", "day"]).count()
  newDF = newDF.select("userId", "count").groupBy("userId").avg().dropDuplicates()
  temp = df.select("userId").dropDuplicates()
  newDF = temp.join(newDF, "userId", "left_outer")
  newDF = newDF.fillna(0).select("userId", F.col("avg(count)").alias("downgradesPerDayAVG"))
  return newDF

In [0]:
def generateDFChurn(df):
  temp = df.select("userId").filter(sparkifyDF.page.like("Cancellation Confirmation")).dropDuplicates()
  temp = temp.withColumn("churned", F.lit(1))
  temp2 = df.select("userId").dropDuplicates().join(temp, "userId", "left_anti")
  temp2 = temp2.withColumn("churned", F.lit(0))
  newDF = temp.join(temp2, ["userId", "churned"], "outer")
  return newDF

In [0]:
def generateDFTotalFriends(df):
  temp = df.select("userId", "page").filter(F.col("page").like("Add Friend"))
  newDF = temp.groupBy("userId").count()
  temp = df.select("userId").dropDuplicates()
  newDF = temp.join(newDF, "userId", "left_outer")
  newDF = newDF.fillna(0).select("userId", F.col("count").alias("numberOfFriends"))
  return newDF

In [0]:
def generateDFFriendsAddedPerDay(df):
  temp = df.select("userId", "page", "day").filter(F.col("page").like("Add Friend"))
  newDF = temp.groupBy(["userId", "day"]).count()
  newDF = newDF.select("userId", "count").groupBy("userId").avg().dropDuplicates()
  temp = df.select("userId").dropDuplicates()
  newDF = temp.join(newDF, "userId", "left_outer")
  newDF = newDF.fillna(0).select("userId", F.col("avg(count)").alias("friendsAddedPerDayAVG"))
  return newDF

In [0]:
def generateDFNumberThumbsUpPerDay(df):
  temp = df.select("userId", "page", "day").filter(F.col("page").like("Thumbs Up"))
  newDF = temp.groupBy(["userId", "day"]).count()
  newDF = newDF.select("userId", "count").groupBy("userId").avg().dropDuplicates()
  temp = df.select("userId").dropDuplicates()
  newDF = temp.join(newDF, "userId", "left_outer")
  newDF = newDF.fillna(0).select("userId", F.col("avg(count)").alias("thumbsUpPerDayAVG"))
  return newDF

In [0]:
def generateDFNumberThumbsDownPerDay(df):
  temp = df.select("userId", "page", "day").filter(F.col("page").like("Thumbs Down"))
  newDF = temp.groupBy(["userId", "day"]).count()
  newDF = newDF.select("userId", "count").groupBy("userId").avg().dropDuplicates()
  temp = df.select("userId").dropDuplicates()
  newDF = temp.join(newDF, "userId", "left_outer")
  newDF = newDF.fillna(0).select("userId", F.col("avg(count)").alias("thumbsDownPerDayAVG"))
  return newDF

In [0]:
def generateDFNumberOfSongsPerDay(df):
  temp = df.select("userId", "page", "day").filter(F.col("page").like("NextSong"))
  newDF = temp.groupBy(["userId", "day"]).count()
  newDF = newDF.select("userId", "count").groupBy("userId").avg()
  temp = df.select("userId").dropDuplicates()
  newDF = temp.join(newDF, "userId", "left_outer")
  newDF = newDF.fillna(0).select("userId", F.col("avg(count)").alias("songsPerDayAVG"))
  return newDF

In [0]:
def addProportionThumbsUpDown(df):
  newDF = df.withColumn("thumbRate", F.when(df.thumbsDownPerDayAVG != 0, df.thumbsUpPerDayAVG / df.thumbsDownPerDayAVG).otherwise(df.thumbsUpPerDayAVG))
  return newDF

In [0]:
def addProportionSongsAds(df):
  newDF = df.withColumn("songRate", F.when(df.adsPerDayAVG != 0, df.songsPerDayAVG / df.adsPerDayAVG).otherwise(df.songsPerDayAVG))
  return newDF

In [0]:
def generateDFAlternateVariables(df):
  newDF = generateDFAddPlaylistPerDay(df)
  newDF = newDF.join(generateDFAdvertsPerDay(df), "userId", "inner")
  newDF = newDF.join(generateDFAge(df), "userId", "inner")
  newDF = newDF.join(generateDFLevel(df), "userId", "inner")
  newDF = newDF.join(generateDFErrorsPerDay(df), "userId", "inner")
  newDF = newDF.join(generateDFDowngradesPerDay(df), "userId", "inner")
  newDF = newDF.join(generateDFChurn(df), "userId", "inner")
  newDF = newDF.join(generateDFTotalFriends(df), "userId", "inner")
  newDF = newDF.join(generateDFFriendsAddedPerDay(df), "userId", "inner")
  newDF = newDF.join(generateDFNumberThumbsUpPerDay(df), "userId", "inner")
  newDF = newDF.join(generateDFNumberThumbsDownPerDay(df), "userId", "inner")
  newDF = newDF.join(generateDFNumberOfSongsPerDay(df), "userId", "inner")
  newDF = newDF.join(generateDFGender(df), "userId", "inner")
  newDF = addProportionThumbsUpDown(newDF)
  newDF = addProportionSongsAds(newDF)
  return newDF


In [0]:
inputDF = generateDFAlternateVariables(sparkifyDF)

In [0]:
#display(inputDF)

In [0]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans as KMeansPandas
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

In [0]:
pandasDF = inputDF.select("*").toPandas()
headers = inputDF.columns
pandasDF.columns = headers

In [0]:
dfKMeans = pandasDF[inputDF.drop("churned").drop("userId").columns]
dfKMeans.head(10)

Out[25]:

,addPlaylistPerDayAVG,adsPerDayAVG,age,level,errorsPerDayAVG,downgradesPerDayAVG,numberOfFriends,friendsAddedPerDayAVG,thumbsUpPerDayAVG,thumbsDownPerDayAVG,songsPerDayAVG,gender,thumbRate,songRate
0,3.615385,2.000000,65.095058,1,1.5,1.904762,71,2.958333,4.766667,1.681818,93.944444,0,2.834234,46.972222
1,7.000000,11.000000,126.756227,0,1.0,0.000000,7,3.500000,7.500000,1.000000,65.500000,1,7.500000,5.954545
2,1.000000,8.500000,71.872141,0,0.0,0.000000,0,0.000000,2.000000,1.000000,36.000000,0,2.000000,4.235294
3,1.000000,9.250000,65.484688,0,0.0,1.000000,1,1.000000,3.500000,1.500000,59.000000,1,2.333333,6.378378
4,2.700000,3.444444,115.284757,0,1.0,2.000000,10,1.666667,3.636364,1.000000,60.928571,1,3.636364,17.688940
5,2.200000,1.500000,71.552477,1,0.0,1.250000,10,3.333333,2.714286,1.250000,42.888889,1,2.171429,28.592593
6,3.450000,8.750000,69.815289,1,1.5,1.571429,42,3.000000,5.045455,2.000000,100.500000,0,2.522727,11.485714
7,1.000000,11.000000,14.328449,0,0.0,0.000000,3,1.500000,2.000000,1.500000,48.000000,1,1.333333,4.363636
8,2.750000,1.000000,54.564977,1,0.0,2.250000,3,1.000000,3.500000,1.500000,83.000000,0,2.333333,83.000000
9,1.857143,7.111111,74.227535,0,1.0,0.000000,11,2.750000,2.111111,1.200000,34.636364,1,1.759259,4.870739


In [0]:
scaledDF = StandardScaler().fit_transform(dfKMeans)

In [0]:
for i in range(2, 15):
  kMeans = KMeansPandas(init = "random", n_clusters = i, n_init = 11)
  kMeans.fit(scaledDF)
  score = silhouette_score(scaledDF, kMeans.labels_)
  print(i, ": ",score)

2 : 0.24997739543915687
3 : 0.16854573135657572
4 : 0.14083933792758158
5 : 0.1326420975303502
6 : 0.13887036256950355
7 : 0.13312977948152435
8 : 0.12181866602658598
9 : 0.11416001686803523
10 : 0.12232846630525783
11 : 0.11285296054120099
12 : 0.12698182821183537
13 : 0.11754949164647464
14 : 0.11889580024181098

In [0]:
kMeans = KMeansPandas(init = "random", n_clusters = 2, n_init = 6)
kMeans.fit(scaledDF)
labels = kMeans.labels_
pandasDF["clus_km"] = labels
score = silhouette_score(scaledDF, kMeans.labels_)
print(i, ": ",score)
pandasDF.groupby("clus_km").mean()

14 : 0.24997739543915687
Out[29]:

,addPlaylistPerDayAVG,adsPerDayAVG,age,level,errorsPerDayAVG,downgradesPerDayAVG,churned,numberOfFriends,friendsAddedPerDayAVG,thumbsUpPerDayAVG,thumbsDownPerDayAVG,songsPerDayAVG,gender,thumbRate,songRate
clus_km,,,,,,,,,,,,,,,
0,1.705928,3.194992,80.288860,0.277512,0.244019,0.491889,0.196172,5.239234,1.602487,2.503985,1.006039,34.623775,0.459330,2.059766,14.285959
1,3.309534,2.606549,85.056332,0.912134,0.812841,1.882105,0.242678,29.255230,2.915152,5.422675,1.884385,81.779786,0.426778,3.198973,48.049005


In [0]:
kMeans = KMeansPandas(init = "random", n_clusters = 10, n_init = 6)
kMeans.fit(scaledDF)
labels = kMeans.labels_
pandasDF["clus_km"] = labels
score = silhouette_score(scaledDF, kMeans.labels_)
print(i, ": ",score)
pandasDF.groupby("clus_km").mean()

14 : 0.12405662938024993
Out[30]:

,addPlaylistPerDayAVG,adsPerDayAVG,age,level,errorsPerDayAVG,downgradesPerDayAVG,churned,numberOfFriends,friendsAddedPerDayAVG,thumbsUpPerDayAVG,thumbsDownPerDayAVG,songsPerDayAVG,gender,thumbRate,songRate
clus_km,,,,,,,,,,,,,,,
0,2.111285,2.573025,68.698852,0.985075,0.089552,1.103305,0.283582,7.611940,2.069620,3.350952,1.221333,44.057699,0.626866,2.497823,23.144452
1,1.852103,4.260128,71.487353,0.000000,0.325000,0.071875,0.187500,4.762500,1.759494,2.498321,1.128631,33.958555,0.262500,2.124310,8.813429
2,0.845833,1.861788,80.924463,0.062500,0.041667,0.062500,0.104167,1.333333,0.672619,1.410218,0.378472,21.057887,0.645833,1.349702,12.245150
3,3.170803,0.771986,78.603592,0.936170,0.753546,2.052319,0.255319,21.787234,2.737882,5.038264,2.072699,86.855166,0.680851,2.696683,79.840796
4,2.593259,3.469745,141.090098,0.100000,0.700000,1.720079,0.133333,15.866667,2.468611,3.710712,1.616723,55.465507,0.533333,2.443172,18.803735
5,2.960916,3.460686,72.174371,0.935484,1.024194,2.030949,0.354839,22.693548,2.607652,4.262518,2.129790,67.621650,0.322581,2.179031,22.276333
6,4.524252,0.454545,88.892688,1.000000,0.666667,1.823243,0.363636,19.636364,3.453078,7.695280,1.940329,138.377525,0.272727,4.662870,138.377525
7,3.660476,9.986825,81.103265,0.666667,0.500000,0.941429,0.466667,14.733333,3.191325,5.415332,2.151429,71.471251,0.266667,3.108757,7.451903
8,3.749498,2.658205,88.860877,0.883721,1.105094,2.053115,0.162791,68.976744,3.310090,6.408738,1.906319,101.806267,0.372093,3.440361,48.086607


###Preguntas

Cual es el promedio de anuncios al dia que ve un usuario gratis

In [0]:
temp = inputDF.filter(F.col("level") == 0)
temp = temp.select("level", "adsPerDayAVG").groupBy("level").avg()
temp = temp.select(F.col("avg(adsPerDayAVG)").alias("ads")).head()
temp = temp.ads
print(temp)

3.543428534314982

Cual es el promedio de anuncios al dia que ve un usuario pago

In [0]:
temp = inputDF.filter(F.col("level") == 1)
temp = temp.select("level", "adsPerDayAVG").groupBy("level").avg()
temp = temp.select(F.col("avg(adsPerDayAVG)").alias("ads")).head()
temp = temp.ads
print(temp)

2.468292785879187

Cuales son los 10 usuarios mas viejos del sistema

In [0]:
temp = inputDF.select("userId", "age").orderBy(F.desc("age"))
display(temp.take(10))

userId,age
159,390.31975694444446
200045,263.91400462962963
6,256.8010185185185
100026,231.85443287037037
279,226.27150462962962
288,225.24359953703703
217,223.21802083333333
300001,195.08846064814816
198,193.3410300925926
119,192.7933101851852


Cuantas canciones tiene la playlist mas grande del sistema

In [0]:
temp = inputDF.withColumn("songsPlaylist", inputDF.age * inputDF.addPlaylistPerDayAVG)
temp = temp.orderBy(F.desc("songsPlaylist"))
temp = temp.head()
temp = temp.songsPlaylist
print(temp)

948.6765885416667

Cuantas personas agregan al dia los usuarios pagos

In [0]:
temp = inputDF.filter(F.col("level") == 1)
temp = temp.select("level", "friendsAddedPerDayAVG").groupBy("level").avg()
temp = temp.select(F.col("avg(friendsAddedPerDayAVG)").alias("friends")).head()
temp = temp.friends
print(temp)

2.674229606458013

Cuantas personas agregan al dia los usuarios gratis

In [0]:
temp = inputDF.filter(F.col("level") == 0)
temp = temp.select("level", "friendsAddedPerDayAVG").groupBy("level").avg()
temp = temp.select(F.col("avg(friendsAddedPerDayAVG)").alias("friends")).head()
temp = temp.friends
print(temp)

1.7067082796340112

Cuantos amigos tienen en promedio los usuarios que han abandonado el sistema

In [0]:
temp = inputDF.filter(F.col("churned") == 1)
temp = temp.select("churned", "numberOfFriends").groupBy("churned").avg()
temp = temp.select(F.col("avg(numberOfFriends)").alias("friends")).head()
temp = temp.friends
print(temp)

15.525252525252526

Cuantas canciones han escuchado los usuarios del sistema en total

In [0]:
temp = inputDF.withColumn("numCanciones", inputDF.age * inputDF.songsPerDayAVG)
temp = temp.select("numCanciones").groupBy().sum()
temp = temp.select(F.col("sum(numCanciones)").alias("numCanciones")).head()
temp = temp.numCanciones
print(temp)

2239608.7997241104